# Diffusion Equation in 1D

We will solve a diffusion equation in 1D:

$$
\frac{\partial y}{\partial t} = \frac{\partial^2 y}{\partial x^2} -e^{-t}(sin(\pi x) - \pi^2 sin(\pi x)), \qquad \text{where} \quad x  \in [-1,1],  \quad t \in [0,1],
$$

with the Dirichlet boundary conditions and the initial condition:

$$
y(-1, t) = y(1,t) = 0, \qquad  y(x,0) = sin(\pi x).
$$

The reference solution is $y = e^{-t}sin(\pi x)$.

## Implementation and Training

First, we import the libraries:

In [150]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from matplotlib.animation import PillowWriter
import datetime
import math

We define the $sine$ and $euler$ functions and the maximum and minimum values of the x domain:

In [142]:
sin = torch.sin
e = torch.exp
pi = math.pi

x_min = -1.
x_max = 1.

Now, we set the parameters of the neural network: it has a structure with 2 input ($x, t$) and 1 output ($y(x,t)$), 15000 maximum training steps, 3 hidden layers with 32 neurons each, 100 samples and a target minimum loss value of $10^{-4}$.

In [143]:
inputs = 2
outputs = 1
hn_1 = 32
hn_2 = 32
hn_3 = 32
steps = 0
max_steps = 15000
loss = 10
min_loss = 1e-4
log_each = 500
samples = 100
loss_values = []

In this part, we define a new class implementing the activation function $sin(x)$, due to the oscillatory nature of the solution.

In [144]:
class Sine(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, x):
        return torch.sin(x)

The multilayer perceptron (MLP) structure is:

In [145]:
mlp = nn.Sequential(
    nn.Linear(inputs,hn_1),
    Sine(),
    nn.Linear(hn_1, hn_2),
    Sine(),
    nn.Linear(hn_2, hn_3),
    Sine(),
    nn.Linear(hn_3, outputs)
)

optimizer = torch.optim.Adam(mlp.parameters())
criterion = nn.MSELoss()
mlp.train()

Sequential(
  (0): Linear(in_features=2, out_features=32, bias=True)
  (1): Sine()
  (2): Linear(in_features=32, out_features=32, bias=True)
  (3): Sine()
  (4): Linear(in_features=32, out_features=32, bias=True)
  (5): Sine()
  (6): Linear(in_features=32, out_features=1, bias=True)
)

In this section, we define a function that calculates the gradients.

In [146]:
def computeGrads(y, x):
    grads, = torch.autograd.grad(y, x, grad_outputs=y.data.new(y.shape).fill_(1), create_graph=True, only_inputs=True)
    return grads

Next, we define the main training loop and the timer:

In [147]:
starttime_train = datetime.datetime.now()
print('----Training Started----')

while steps < max_steps and loss > min_loss:
    x = (x_max - x_min)*torch.rand(samples) + x_min
    t = torch.rand(samples)
    X = torch.stack([x, t], axis=-1)
    X.requires_grad = True
    Y = mlp(X)
    grads = computeGrads(Y, X)
    dydx = grads[:, :1]
    dydt = grads[:, 1:]
    grads2 = computeGrads(dydx, X)
    d2ydx2 = grads2[:, :1]
    ode_loss = criterion(dydt, d2ydx2 - (e(-t)*(sin(pi*x) - (pi**2)*sin(pi*x))).unsqueeze(1))

    #initial condition
    x = (x_max - x_min)*torch.rand(samples) + x_min
    t_0 = torch.zeros(samples)
    X_initial = torch.stack([x, t_0], axis=-1)
    Y_initial = mlp(X_initial)
    y_initial = sin(pi*x).unsqueeze(1)
    ic_loss = criterion(Y_initial, y_initial)

    #boundary conditions x=-1
    x_1 = -torch.ones(samples)
    t = torch.rand(samples)
    X_boundary_1 = torch.stack([x_1, t], axis=-1)
    Y_boundary_1 = mlp(X_boundary_1)
    zero = torch.zeros(samples, 1)
    bc_1_loss = criterion(Y_boundary_1, zero)
    
    #boundary conditions x=1
    x1 = torch.ones(samples)
    X_boundary1 = torch.stack([x1, t], axis=-1)
    Y_boundary1 = mlp(X_boundary1)
    bc1_loss = criterion(Y_boundary1, zero)

    bc_loss = bc_1_loss + bc1_loss

    optimizer.zero_grad()
    loss = ode_loss + ic_loss  + bc_loss
    loss.backward()
    optimizer.step()

    loss_values.append(loss.detach().numpy())

    if steps % log_each == 0:
        print(f'Step:{steps}| ode_loss {ode_loss.item():.5f} ic_loss {ic_loss.item():.5f} bc_loss {bc_loss.item():.5f}')

    steps+=1

endtime_train = datetime.datetime.now()
train_time = endtime_train - starttime_train
train_time_formatted = train_time.seconds + train_time.microseconds / 1e6
print('---Training Finished---')

print(f'Training Duration: {steps} steps in {train_time_formatted:.3f} seconds')

----Training Started----
Step:0| ode_loss 17.13283 ic_loss 0.62717 bc_loss 0.09392
Step:500| ode_loss 0.12982 ic_loss 0.00267 bc_loss 0.02203
Step:1000| ode_loss 0.01297 ic_loss 0.00177 bc_loss 0.00288
Step:1500| ode_loss 0.00718 ic_loss 0.00060 bc_loss 0.00100
Step:2000| ode_loss 0.00517 ic_loss 0.00022 bc_loss 0.00028
Step:2500| ode_loss 0.00540 ic_loss 0.00031 bc_loss 0.00050
Step:3000| ode_loss 0.00266 ic_loss 0.00024 bc_loss 0.00025
Step:3500| ode_loss 0.00518 ic_loss 0.00028 bc_loss 0.00084
Step:4000| ode_loss 0.00073 ic_loss 0.00011 bc_loss 0.00027
Step:4500| ode_loss 0.00064 ic_loss 0.00009 bc_loss 0.00015
Step:5000| ode_loss 0.00063 ic_loss 0.00007 bc_loss 0.00030
Step:5500| ode_loss 0.00082 ic_loss 0.00004 bc_loss 0.00021
Step:6000| ode_loss 0.00023 ic_loss 0.00008 bc_loss 0.00010
Step:6500| ode_loss 0.00020 ic_loss 0.00004 bc_loss 0.00007
Step:7000| ode_loss 0.00029 ic_loss 0.00005 bc_loss 0.00005
Step:7500| ode_loss 0.00047 ic_loss 0.00004 bc_loss 0.00007
Step:8000| ode_los

## Visualization

To visualize the model solution, we generate the output of the trained model and display it as a GIF alongside the reference solution.

In [ ]:
x = torch.linspace(x_min, x_max, samples)
t = torch.linspace(0, 1, samples)

def sol(x, t):
    return e(-t)*sin(pi*x)

y = [] 
ref_sol = []
time = []

for t_ in t:
    with torch.no_grad():
        X = torch.stack([x, torch.ones(samples)*t_], axis=-1)
        Y = mlp(X)
    y.append(Y.detach().numpy())
    ref_sol.append(sol(x, t_))
    time.append(t_)

def update(i):
    ax.clear()
    ax.plot(x, y[i], '.-', label = 'PINN')
    ax.plot(x, ref_sol[i], '-', label = 'Reference')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y(x,t)$')
    ax.set_title(f'$t = {time[i]:.3f}$')
    ax.set_ylim(-1.5, 1.5)
    ax.grid(True)
    ax.legend()
    return ax

fig = plt.figure(dpi=100)
ax = plt.subplot(1,1,1)
anim = animation.FuncAnimation(fig, update, frames=len(y), interval=200)
plt.close(fig)

<a href="https://raw.githubusercontent.com/munozmfrancisco/PINNs/main/GIFs/Forward/diffusion.gif" target="_blank">
    Animation
</a>